In [13]:
from flask import Flask
from sqlalchemy import Column, String, Integer, ForeignKey, DateTime
from sqlalchemy.orm import relationship
from flask_sqlalchemy import SQLAlchemy

# make app and db setup

In [14]:
db = SQLAlchemy()

In [15]:
app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///'
db.init_app(app)

# Models

In [ ]:
class User(db.Model):
  __tablename__ = 'users'
  id = Column(Integer, primary_key=True, autoincrement=True)
  email = Column(String, unique=True)
  password = Column(String)
  type = Column(String)
  full_name = Column(String)
  address = Column(String)
  pin_code = Column(String)
  city_id = Column(Integer, ForeignKey('cities.id'))
  city = relationship('City', back_populates='users')
  customer = relationship('Customer', back_populates='user', uselist=False)
  professional = relationship('Professional', back_populates='user', uselist=False)
  admin = relationship('Admin', back_populates='user', uselist=False)

In [ ]:
class City(db.Model):
  __tablename__ = 'cities'
  id = Column(Integer, primary_key=True, autoincrement=True)
  name = Column(String)
  users = relationship('User', back_populates='city')

In [18]:
class Customer(db.Model):
  __tablename__ = 'customers'
  id = Column(Integer, primary_key=True, autoincrement=True)
  user_id = Column(Integer, ForeignKey('users.id'))
  user = relationship('User', back_populates='customer')
  service_requests = relationship('ServiceRequest', back_populates='customer')


class Professional(db.Model):
  __tablename__ = 'professionals'
  id = Column(Integer, primary_key=True, autoincrement=True)
  user_id = Column(Integer, ForeignKey('users.id'))
  service_category_id = Column(Integer, ForeignKey('service_categories.id'))
  user = relationship('User', back_populates='professional')
  service_category = relationship('ServiceCategory', back_populates='professionals')
  service_requests = relationship('ServiceRequest', back_populates='professional')


class Admin(db.Model):
  __tablename__ = 'admins'
  id = Column(Integer, primary_key=True, autoincrement=True)
  user_id = Column(Integer, ForeignKey('users.id'))
  user = relationship('User', back_populates='admin')

In [19]:
class ServiceCategory(db.Model):
  __tablename__ = 'service_categories'
  id = Column(Integer, primary_key=True, autoincrement=True)
  name = Column(String)
  professionals = relationship('Professional', back_populates='service_category')
  services = relationship('Service', back_populates='service_category')


class Service(db.Model):
  __tablename__ = 'services'
  id = Column(Integer, primary_key=True, autoincrement=True)
  name = Column(String)
  price = Column(Integer)
  service_category_id = Column(Integer, ForeignKey('service_categories.id'))
  service_category = relationship('ServiceCategory', back_populates='services')
  service_requests = relationship('ServiceRequest', back_populates='service')


class ServiceRequest(db.Model):
  __tablename__ = 'service_requests'
  id = Column(Integer, primary_key=True, autoincrement=True)
  customer_id = Column(Integer, ForeignKey('customers.id'))
  service_id = Column(Integer, ForeignKey('services.id'))
  description = Column(String)
  professional_id = Column(Integer, ForeignKey('professionals.id'))
  status = Column(String)
  created_date = Column(DateTime)
  booking_date = Column(DateTime)
  ratings = Column(Integer)
  remarks = Column(String)

  customer = relationship('Customer', back_populates='service_requests')
  service = relationship('Service', back_populates='service_requests')
  professional = relationship('Professional', back_populates='service_requests')

# make tables

In [20]:
with app.app_context():
  db.create_all()

# populate data

In [21]:
with app.app_context():
  db.session.add(User(
      email='user1@example.com',
      password='12345',
      type='customer',
      full_name='vidhatri',
      address='ramanagar',
      pin_code='123456',
      city_id=1,
      customer=Customer()
  ))
  db.session.add(User(
      email='user2@example.com',
      password='12345',
      type='customer',
      full_name='rahul',
      address='ramanagar',
      pin_code='123456',
      city_id=1,
      customer=Customer()
  ))
  db.session.add(User(
      email='user3@example.com',
      password='12345',
      type='professional',
      full_name='harikesh',
      address='ramanagar',
      pin_code='123456',
      city_id=2,
      professional=Professional(service_category_id=1)
  ))

  db.session.add(City(name='hbh'))
  db.session.add(City(name='albd'))
  db.session.add(ServiceCategory(name='Plumbing services', services=[
      Service(name='Tap repair'),
      Service(name='Heater setup'),
      Service(name='Pipe leakage')
  ]))
  db.session.commit()

  users = User.query.all()
  cities = City.query.all()
  print(users)
  print(cities)

[<User 1>, <User 2>, <User 3>]
[<City 1>, <City 2>]


# testing/querying

In [22]:
with app.app_context():
  customers = Customer.query.all()
  professionals = Professional.query.all()
  service_categories = ServiceCategory.query.all()
  services = Service.query.all()

In [23]:
# name of user with email = user1@example.com
with app.app_context():
  user = User.query.filter_by(email='user1@example.com').first()
  print(f'{user.full_name} from {user.city.name}')

vidhatri from hbh


In [24]:
# user from city id=1
with app.app_context():
  city = City.query.filter_by(id=1).first()
  for user in city.users:
    print(f'{user.full_name} from {city.name}')

vidhatri from hbh
rahul from hbh
